In [1]:
import numpy as np
import pandas as pd
import openpyxl
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

def calculate_implied_yield_from_futures(self, benchmark_yields) -> pd.Series:
    """
    Calculate the average future price using benchmark yields and hedge ratios.
    """
    return (25 / benchmark_yields[self.benchmark_instruments] * self.hedge_ratios).sum(axis=1)

# Load future data
# df_future = pd.read_csv('data/euribor_future_data.csv')
# df_future.head()


In [ ]:
# Load swap yield pivot data
df_swap_yield = pd.read_csv('data\\swap_yield_pivot.csv', index_col=0)
print(len(df_swap_yield))
df_swap_yield.head()

# Remove all rows starting from '2025-05-20 06:00:00'
df_swap_yield = df_swap_yield[df_swap_yield.index < '2025-05-20 06:00:00']
df_swap_yield.iloc[-3:]

# Load future yield pivot data
df_future_yield = pd.read_csv('data/future_yield_pivot.csv', index_col=0)
df_future_yield.head()

In [ ]:
# [7] Load hedge ratios
with open('data\\hedge_ratio.pkl', 'rb') as fp:
    hedge_ratios = pickle.load(fp)

# [8] Mapping for swap fido converter
swap_fido_converter = {
    427868: 'SW1Y6M',
    427861: 'SW2Y6M',
    427862: 'SW3Y6M',
    427865: 'SW4Y6M',
    # 427863: 'SW5Y6M'
}

In [ ]:
def compute_convexity_vectorized():
    convexities = {fido: [] for fido in swap_fido_converter.keys()}
    
    for d in df_swap_yield.index:
        date = d[:10]
        swap_hedges = hedge_ratios[date]
        swap_yields = df_swap_yield.loc[d]
        
        # Convert df_future_yield to have the correct row for this timestamp
        future_yield_row = df_future_yield.loc[d]
        
        for fido, future_weights in swap_hedges.items():
            # Convert future_weights dict to Series
            weights_series = pd.Series(future_weights)
            
            # Ensure only available ERs are considered (align weights and yield)
            common_ers = weights_series.index.intersection(future_yield_row.index)
            weights_series = weights_series.loc[common_ers]
            future_yield_row_aligned = future_yield_row.loc[common_ers]
            
            # Vectorized dot product to compute implied yield
            implied_yield = (weights_series * future_yield_row_aligned).sum()
            
            # Subtract to get convexity
            swap_yield = swap_yields.loc[str(fido)]
            convexity = swap_yield - implied_yield
            
            convexities[fido].append(convexity)
    
    return convexities


In [ ]:
def compute_convexity():
    convexities = {}
    
    for d in df_swap_yield.index:
        # d is datetime in string format: e.g. '2024-12-16 07:00:00'
        date = d[:10]  # e.g. '2024-12-16'
        swap_hedges = hedge_ratios[date]
        swap_yields = df_swap_yield.loc[d]
        
        for fido in swap_hedges:
            if fido not in convexities:
                convexities[fido] = []

            future_weights = swap_hedges[fido]
            swap_yield = swap_yields.loc[str(fido)]
            
            # Method 1: Using matrix multiplication
            implied_yield = df_future_yield.mul(pd.Series(future_weights)).sum(axis=1)
            print(implied_yield)

            # Method 2: Manual weighted sum (like in the image)
            implied_yield_manual = 0
            for er in future_weights:
                implied_yield_manual += future_weights[er] * df_future_yield[er].loc[d]
            print(implied_yield_manual)
            
            # Store convexity
            convexities[fido].append(swap_yield - implied_yield_manual)

            # Break after first fido per date (based on screenshot)
            break
        # Break after first date (based on screenshot)
        break

    return convexities


In [ ]:
# full vectorized workflow

with open('data\\hedge_ratio.pkl', 'rb') as fp:
    hedge_ratios = pickle.load(fp)

# 3️⃣ Build a weights matrix (fidos x ERs) for each date
# Assume: hedge_ratios[date] = {fido: {ER: weight, ...}, ...}

# Get list of unique fidos
all_fidos = set()
for date_hedges in hedge_ratios.values():
    all_fidos.update(date_hedges.keys())

# Initialize convexities
convexities = {fido: [] for fido in all_fidos}

# Loop over each timestamp
for d in df_swap_yield.index:
    date = d[:10]
    
    # Check if date exists in hedge_ratios
    if date not in hedge_ratios:
        continue
    
    # Build weights DataFrame for the current date
    swap_hedges = hedge_ratios[date]
    weights_data = pd.DataFrame.from_dict(swap_hedges, orient='index').fillna(0)
    
    # Align weights and future_yield columns
    common_ers = weights_data.columns.intersection(df_future_yield.columns)
    weights_data = weights_data[common_ers]
    future_yield_row = df_future_yield.loc[d, common_ers]
    
    # 4️⃣ Compute implied yields for all fidos at once
    # future_yield_row: (ERs,)
    # weights_data: (fidos, ERs)
    # Compute dot product row-wise
    implied_yields = weights_data.dot(future_yield_row)
    
    # 5️⃣ Compute convexities: swap_yield - implied_yield
    swap_yields_row = df_swap_yield.loc[d]
    
    for fido in implied_yields.index:
        # Ensure swap_yield is aligned (string keys)
        swap_yield = swap_yields_row.get(str(fido), np.nan)
        if pd.notna(swap_yield):
            convexity = swap_yield - implied_yields.loc[fido]
            convexities[fido].append(convexity)

# 6️⃣ Example: Show convexities for the first few fidos
for fido, conv_list in convexities.items():
    print(f"Fido: {fido}")
    print(conv_list[:5])  # first 5 entries
    print("="*40)


In [ ]:

# Checking data length
print(len(df_swap_yield), len(df_future_yield))  # (126105, 126105)

# Show swap_fido_converter again
print(swap_fido_converter)

# Example future yield dataframe (mocked based on the screenshots)
data = {
    'datetime': [
        '2024-12-16 07:00:00',
        '2024-12-16 07:00:30',
        '2024-12-16 07:01:00',
        '2024-12-16 07:01:30',
        '2024-12-16 07:02:00'
    ],
    'ER1': [229.50, 229.75, 229.75, 229.75, 229.75],
    'ER10': [205.25, 205.00, 205.00, 205.25, 205.25],
    'ER11': [207.25, 207.25, 207.25, 207.25, 207.25],
    'ER12': [209.75, 209.25, 209.75, 209.75, 209.75],
    'ER13': [211.75, 211.75, 211.75, 212.00, 212.00],
    'ER14': [214.25, 214.25, 214.25, 214.25, 214.25],
    'ER15': [216.75, 216.50, 216.75, 216.75, 216.75],
    'ER16': [218.75, 218.75, 218.75, 218.75, 219.00],
    'ER17': [221.25, 221.25, 221.25, 221.25, 221.50]
}
df_future_yield = pd.DataFrame(data)
df_future_yield.set_index('datetime', inplace=True)
print(df_future_yield.head())


In [ ]:
# convexity_df = compute_convexity_vectorized()
# convexity_df.set_index('datetime').plot(figsize=(12, 6), grid=True)